In [1]:
import os
import json
from openai import OpenAI
from dotenv import load_dotenv
load_dotenv()

client = OpenAI(
  api_key=os.environ['HF_API_KEY'],  # this is also the default, it can be omitted
  base_url = "https://router.huggingface.co/v1"
  )

deployment="openai/gpt-oss-20b:fireworks-ai"

In [ ]:
## VERSION 1 : function calling - local function

# define tools 
function_schema = { 
    "type":"function",
    "name": "get_word_of_the_day",
    "description" : "Retrieves the word of the day based in a given language",
    "parameters" : {
        "type" : "object",
        "properties" : {
            "language" : {
                "type": "string",
                "enum" : ["french", "english", "italian"],
                "description" : "The language"
                } 
            },
            "required": ["language"],
            "additionalProperties" : False
        },
    "strict": True
}


# Harmony-style system message
system_prompt = f"""
You are a function-calling assistant.
You have the following function available:

Name: {function_schema['name']}
Description: {function_schema['description']}
Parameters: {json.dumps(function_schema['parameters'], indent=2)}

When a user asks something that matches this function, respond ONLY in valid JSON with:
{{
  "name": "<function_name>",
  "arguments": {{...}}
}}
Do not add extra text or explanation.
"""

def get_word_of_the_day(language: str) -> str:
    word = ""
    if language ==  'english':
        word = "GOOD"
    elif language == 'french':
        word = "SALUT"
    elif language == 'italian':
        word = "CIAO"
    return f"In {language}: the word of the day is '{word}'"



# 1st prompt with tools

prompt = "Qual e la parola del giorno ?"

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": prompt}
] 

response =  client.chat.completions.create(   
    model=deployment,                                         
    messages=messages,
    max_tokens=200
)



In [43]:
## VERSION 2 : function calling - API CALL

import requests

# define tools 
function_schema = { 
    "type":"function",
    "name": "search_courses",
    "description" : "Retrieves a selection of Microsoft courses based on the level, role and product searched by user",
    "parameters" : {
        "type" : "object",
        "properties" : {
            "level" : {
                "type": "string",
                "enum" : ["beginner", "intermediate", "advanced"],
                "description" : "The level of the course"
                },
            "role" : {
                "type": "string",
                "enum": ["business-analyst", "developer", "ai-engineer"],
                "description": "The role to learn in the course."
                },
            "product" : {
                "type":"string",
                "enum": ["dotnet","microsoft-office", "sql-server"],
                "description": "The product to learn"
            } 
            },
            "required": ["level"],
            "additionalProperties" : False
        },
    "strict": True
}


# Harmony-style system message
system_prompt = f"""
You are a function-calling assistant.
You have the following function available:

Name: {function_schema['name']}
Description: {function_schema['description']}
Parameters: {json.dumps(function_schema['parameters'], indent=2)}

When a user asks something that matches this function, respond ONLY in valid JSON with:
{{
  "name": "<function_name>",
  "arguments": {{...}}
}}
Do not add extra text or explanation.
"""

def search_courses(**args) -> str:
    url = "https://learn.microsoft.com/api/catalog/"

    response = requests.get(url, params=args)
    courses = response.json()["modules"]
    results = []
    for course in courses[:5]:
        title = course["title"]
        url = course["url"]
        results.append({title: title, url:url})
    return str(results)



# 1st prompt with tools

prompt = "I just started sql, suggest me a Microsoft course"

messages = [
    {"role": "system", "content": system_prompt},
    {"role": "user", "content": prompt}
] 

response =  client.chat.completions.create(   
    model=deployment,                                         
    messages=messages,
)

In [47]:
response_message

{'name': 'search_courses',
 'arguments': {'level': 'beginner', 'product': 'sql-server'}}

In [45]:
response_message = json.loads(response.choices[0].message.content)
if response_message['name']:
    print(f"Recommended Function Call : {response_message['name']}")

    available_functions = {
        'get_word_of_the_day': get_word_of_the_day,
        'search_courses': search_courses
    }

    function_to_call = available_functions[response_message['name']]
    function_args = response_message['arguments']

    function_response = function_to_call(**function_args)
    print(function_response)




Recommended Function Call : search_courses
[{'Design a Performant Data Model in Azure SQL Database with Visual Studio Code': 'Design a Performant Data Model in Azure SQL Database with Visual Studio Code', 'https://learn.microsoft.com/en-us/training/modules/design-a-data-model-with-ads/?WT.mc_id=api_CatalogApi': 'https://learn.microsoft.com/en-us/training/modules/design-a-data-model-with-ads/?WT.mc_id=api_CatalogApi'}, {'Understand hybrid data platform on SQL Server 2022': 'Understand hybrid data platform on SQL Server 2022', 'https://learn.microsoft.com/en-us/training/modules/understand-hybrid-data-platform-sql-server-2022/?WT.mc_id=api_CatalogApi': 'https://learn.microsoft.com/en-us/training/modules/understand-hybrid-data-platform-sql-server-2022/?WT.mc_id=api_CatalogApi'}, {'Sort and filter results in T-SQL': 'Sort and filter results in T-SQL', 'https://learn.microsoft.com/en-us/training/modules/sort-filter-queries/?WT.mc_id=api_CatalogApi': 'https://learn.microsoft.com/en-us/trainin

In [50]:
messages

[{'role': 'system',
  'content': '\nYou are a function-calling assistant.\nYou have the following function available:\n\nName: search_courses\nDescription: Retrieves a selection of Microsoft courses based on the level, role and product searched by user\nParameters: {\n  "type": "object",\n  "properties": {\n    "level": {\n      "type": "string",\n      "enum": [\n        "beginner",\n        "intermediate",\n        "advanced"\n      ],\n      "description": "The level of the course"\n    },\n    "role": {\n      "type": "string",\n      "enum": [\n        "business-analyst",\n        "developer",\n        "ai-engineer"\n      ],\n      "description": "The role to learn in the course."\n    },\n    "product": {\n      "type": "string",\n      "enum": [\n        "dotnet",\n        "microsoft-office",\n        "sql-server"\n      ],\n      "description": "The product to learn"\n    }\n  },\n  "required": [\n    "level"\n  ],\n  "additionalProperties": false\n}\n\nWhen a user asks someth

In [49]:

tool_call = {
    "role": response.choices[0].message.role,
    "function_call": {
        "name": response_message['name'],
        "arguments" : response_message['arguments']
    },
    "content": None
}

tool_call_output = {
    "role": "function",
    "name": response_message['name'],
    "content": function_response
}


messages.append(tool_call)
messages.append(tool_call_output)

second_response =  client.chat.completions.create(   
    model=deployment,                                         
    messages=messages,
)

BadRequestError: Error code: 400 - {'error': {'object': 'error', 'type': 'invalid_request_error', 'code': 'invalid_request_error', 'message': "Extra inputs are not permitted, field: 'messages[2].function_call'"}}